In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install sentence_transformers
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sentence_transformers import SentenceTransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=dd4f940446970cd5c70c4c21ea735a4367b0d3a39d8f3efd873694de41b094ed
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")
%cd "./drive/My Drive"

Mounted at /content/drive/
/content/drive/My Drive


In [ ]:
student = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)
tokenizer = T5Tokenizer.from_pretrained("t5-small")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
teacher = T5ForConditionalGeneration.from_pretrained("t5-large").to(device)
teacher.load_state_dict(torch.load("./t5-large28.pt", map_location=device)["model_state_dict"])
tokenizer_t = T5Tokenizer.from_pretrained("t5-large")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
comparer = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2').to(device)

In [ ]:
student = student.to(device)
teacher = teacher.to(device)
comparer = comparer.to(device)

In [ ]:
from tqdm import tqdm

In [ ]:
def make_label(model, X, tokenizer):
  # Tokenize the input text
  inputs = tokenizer.encode("simplify this to english that is easier to understand: " + X, return_tensors="pt")

  # Generate output from the model
  outputs = model.generate(inputs.to(device), num_beams=4, early_stopping=True, max_new_tokens=4000)

  # Decode the output tokens to text
  output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  return (output_text)

In [ ]:
def make_label_unprompted(model, X, tokenizer):
  # Tokenize the input text
  inputs = tokenizer.encode(X, return_tensors="pt")

  # Generate output from the model
  outputs = model.generate(inputs.to(device), num_beams=4, early_stopping=True, max_new_tokens=4000)

  # Decode the output tokens to text
  output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  return (output_text)

In [ ]:
def train(student,
          teacher,
          X,
          y,
          loss_fn,
          optimizer,
          device):

  student.train()
  teacher.eval()
  train_loss = 0

  for i in tqdm(range(len(X))):
    y_student = make_label(student, X[i], tokenizer)
    # y_teacher = make_label(teacher, X[i], tokenizer_t)

    embeddings1 = comparer.encode(y_student, convert_to_tensor=True).requires_grad_()
    embeddings2 = comparer.encode(y[i], convert_to_tensor=True)

    loss = loss_fn(embeddings1, embeddings2)
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  train_loss = train_loss / len(X)
  return(train_loss)

In [ ]:
def train_unprompt(student,
                   teacher,
                   X,
                   y,
                   loss_fn,
                   optimizer,
                   device):

  student.train()
  teacher.eval()
  train_loss = 0

  for i in tqdm(range(len(X))):
    y_student = make_label_unprompted(student, X[i], tokenizer)
    # y_teacher = make_label(teacher, X[i], tokenizer_t)

    embeddings1 = comparer.encode(y_student, convert_to_tensor=True).requires_grad_()
    embeddings2 = comparer.encode(y[i], convert_to_tensor=True)

    loss = loss_fn(embeddings1, embeddings2)
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  train_loss = train_loss / len(X)
  return(train_loss)

In [ ]:
df = pd.read_csv("./processed_train_data.csv")

In [ ]:
X = df["Expert"].values

In [ ]:
y = []
for x in X:
  y.append(make_label(teacher, x, tokenizer_t))

In [ ]:
labels = pd.DataFrame({"labels" : y})
labels.to_csv("./teacher_labels_unprompted.csv", index=False)

In [ ]:
y = df["Simple"].values

In [ ]:
loss_fn = nn.MSELoss();
optimizer = torch.optim.Adam(student.parameters())

In [ ]:
loss_history = []
for epoch in range(3):
  loss = train(student, teacher, X, y, loss_fn, optimizer, device)
  loss_history.append(loss)
  torch.save({
      "epoch": epoch,
      "model_state_dict": student.state_dict(),
      "optimizer_state_dict": optimizer.state_dict(),
      "train_loss_history": loss_history
  }, "./student_actual.pth")
  print(loss)

100%|██████████| 1397/1397 [13:04<00:00,  1.78it/s]


0.001393571556453139


100%|██████████| 1397/1397 [12:39<00:00,  1.84it/s]


0.0013895321395465151


100%|██████████| 1397/1397 [12:48<00:00,  1.82it/s]


0.0014124066452579087


In [ ]:
for i in range(len(y)):
  print(i)
  print(y[i])

0
75-90 % of the affected people have mild intellectual disability.
1
. A tightness or tingling in the chest or abdomen, numbness in the feet, and difficulty voiding., and difficulty urinating.. A bandlike tightness around the chest or abdomen, weakness, tingling,,., or abdomen,ness, tingling, and numbness in the feet develop over hours toa few days. abdomen. abdomen. days.a few days. develop.
2
.. A damaged or missing ossicular chain may be repaired during tympanoplasty. may be repaired during tympanoplasty. A damaged or missing ossicular chain may be repaired during tympanoplasty. A disrupted or missing ossicular chain may be repaired during tympanoplasty.p.oplasty.cular chain or both.icular chain. duringoplasty...
3
(halos and starbursts) from bright lights or from car headlights while driving at night.... A posterior subcapsular cataract disproportionately affects vision because the opacity is located at the crossing point of incoming light rays. Such cataracts reduce visual acuity

In [ ]:
val = 5
test = make_label_unprompted(student, X[val], tokenizer)
test_b = y[val]

In [ ]:
print(X[val])
print("==============================")
print(test)
print("==============================")
print(test_b)

About 25 % of men have minimal symptoms.
Etwa 25 % of men men have minimal symptoms.
Some men (about 25 %) have minimal symptoms.
